In [584]:
import numpy as np

nmax = 4
arr = np.arange(nmax*nmax, dtype=np.float64).reshape(nmax, nmax)



In [585]:
def one_energy_fast(arr,ix,iy,nmax):
    
    ixp = (ix+1)%nmax # These are the coordinates
    ixm = (ix-1)%nmax # of the neighbours
    iyp = (iy+1)%nmax # with wraparound
    iym = (iy-1)%nmax #

    xy_arr = np.array([arr[ixp,iy], arr[ixm,iy], arr[ix,iyp], arr[ix,iym]])
    ang = arr[ix, iy] - xy_arr
    en = np.sum(0.5*(1.0 - 3.0*np.pow(np.cos(ang),2)))
    return en

#
# Add together the 4 neighbour contributions
# to the energy
#
'''
    ang = arr[ix,iy]-arr[ixp,iy]
    en += 0.5*(1.0 - 3.0*np.cos(ang)**2)
    ang = arr[ix,iy]-arr[ixm,iy]
    en += 0.5*(1.0 - 3.0*np.cos(ang)**2)
    ang = arr[ix,iy]-arr[ix,iyp]
    en += 0.5*(1.0 - 3.0*np.cos(ang)**2)
    ang = arr[ix,iy]-arr[ix,iym]
    en += 0.5*(1.0 - 3.0*np.cos(ang)**2)

    return en
'''

def one_energy(arr,ix,iy,nmax):

    en = 0.0
    ixp = (ix+1)%nmax # These are the coordinates
    ixm = (ix-1)%nmax # of the neighbours
    iyp = (iy+1)%nmax # with wraparound
    iym = (iy-1)%nmax #

    '''
    xy_arr = np.array([arr[ixp,iy], arr[ixm,iy], arr[ixm,iy], arr[ix,iym]])
    en = 0.5*(1.0 - 3.0*np.cos(arr[ix, iy] - xy_arr)**2)
    return np.sum(en)
    '''
#
# Add together the 4 neighbour contributions
# to the energy
#

    ang = arr[ix,iy]-arr[ixp,iy]
    en += 0.5*(1.0 - 3.0*np.cos(ang)**2)
    ang = arr[ix,iy]-arr[ixm,iy]
    en += 0.5*(1.0 - 3.0*np.cos(ang)**2)
    ang = arr[ix,iy]-arr[ix,iyp]
    en += 0.5*(1.0 - 3.0*np.cos(ang)**2)
    ang = arr[ix,iy]-arr[ix,iym]
    en += 0.5*(1.0 - 3.0*np.cos(ang)**2)

    return en


In [588]:
ran = np.random.randint(0,nmax, size = 2)


In [589]:
timeit(one_energy(arr, ran[0], ran[1], nmax))

3.51 μs ± 137 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [590]:
timeit(one_energy_fast(arr, ran[0], ran[1], nmax))

5.2 μs ± 144 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [ ]:
#my 'fast' numpy version actually ended up being slower
one_energy_fast(arr, ran[0], ran[1], nmax)

np.float64(-0.15752969446636644)

In [592]:
one_energy(arr, ran[0], ran[1], nmax)

np.float64(-0.15752969446636644)

In [594]:
grid = np.mgrid[0:nmax,0:nmax].reshape(2,-1).T

def Qab_slow():
    Qab = np.zeros((3,3))
    delta = np.eye(3,3)
    #
    # Generate a 3D unit vector for each cell (i,j) and
    # put it in a (3,i,j) array.
    #
    lab = np.vstack((np.cos(arr),np.sin(arr),np.zeros_like(arr))).reshape(3,nmax,nmax)
    for a in range(3):
        for b in range(3):
            for i, j in grid:
                    Qab[a,b] += 3*lab[a,i,j]*lab[b,i,j] - delta[a,b]

    #print(Qab)



In [598]:
def Qab_fast():
    lab = np.vstack((np.cos(arr),np.sin(arr),np.zeros_like(arr))).reshape(3,nmax,nmax)
    labmult = np.zeros([3, 3])
    delta = np.eye(3,3)
    for a in range(3):
        for b in range(3):
            labmult[a,b] += np.sum(3*lab[a,:,:]*lab[b,:,:] - delta[a,b])

    #print(labmult)

#This version is far faster than the loop version

In [596]:
timeit(Qab_slow())

142 μs ± 2.04 μs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [597]:
timeit(Qab_fast())

35.2 μs ± 500 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [ ]:
def all_energy_slow(arr,nmax):
    """
    Arguments:
	  arr (float(nmax,nmax)) = array that contains lattice data;
      nmax (int) = side length of square lattice.
    Description:
      Function to compute the energy of the entire lattice. Output
      is in reduced units (U/epsilon).
	Returns:
	  enall (float) = reduced energy of lattice.
    """
    enall = 0.0
    for i in range(nmax):
        for j in range(nmax):
            enall += one_energy(arr,i,j,nmax)
    return enall


In [ ]:
def all_energy_fast(arr,nmax, grid):
    """
    Arguments:
	  arr (float(nmax,nmax)) = array that contains lattice data;
      nmax (int) = side length of square lattice.
    Description:
      Function to compute the energy of the entire lattice. Output
      is in reduced units (U/epsilon).
	Returns:
	  enall (float) = reduced energy of lattice.
    """
    enall = np.sum(one_energy(arr, grid[:,0], grid[:,1], nmax))
    return enall

#faster with no loops

In [582]:
all_energy_slow(arr, nmax)

np.float64(-19.052271888752816)

In [583]:
all_energy_fast(arr,nmax,grid)

np.float64(-19.052271888752813)

In [599]:
timeit(all_energy_slow(arr, nmax))

52.8 μs ± 2.7 μs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [600]:
timeit(all_energy_fast(arr,nmax,grid))

21.5 μs ± 494 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [522]:
def MC_step(arr, nmax):

    accept = 0
    
    xran = np.random.randint(0,high=nmax, size=(nmax,nmax))
    yran = np.random.randint(0,high=nmax, size=(nmax,nmax))
    #aran = np.random.normal(scale=0.6, size=(nmax,nmax))
    for ix, iy in grid:
        ang = aran[ix,iy]
        en0 = one_energy(arr,ix,iy,nmax)
        arr[ix,iy] += ang
        en1 = one_energy(arr,ix,iy,nmax)
        if en1<=en0:
            accept += 1
        else:
        # Now apply the Monte Carlo test - compare
        # exp( -(E_new - E_old) / T* ) >= rand(0,1)
            boltz = np.exp( -(en1 - en0) / 0.5 )

            if boltz >= np.random.uniform(0.5, 0.5):
                accept += 1
            else:
                arr[ix,iy] -= ang

    print(arr)
    return accept/(nmax*nmax)

In [ ]:


def MC_step_fast(arr, nmax):
    checkerboard = np.indices([nmax, nmax]).sum(axis=0) % 2
    #aran = np.random.normal(scale=0.6, size=(nmax,nmax))
    boltz = np.random.uniform(0.0,1.0, size=(nmax,nmax))
    accept = 0.0

    #loops over the two halves of the checkerboard
    for i in range(2):
        
        arr_copy = arr.copy()

        en0 = one_energy(arr, grid[:,0], grid[:,1], nmax)
        #only adds the random angle to half the lattice
        arr[checkerboard == i] += aran[checkerboard == i]
        en1 = one_energy(arr, grid[:,0], grid[:,1], nmax)
        #adds one for each energy below the previous energy

        #creates new energies for the checkerboard
        en0_checker = en0.reshape(nmax, nmax)[checkerboard == i]
        en1_checker = en1.reshape(nmax, nmax)[checkerboard == i]
        accept += np.sum(en1_checker <= en0_checker)
        
        #calculates boltz and compares it to a random number
        boltz = np.exp(-(en1_checker[en1_checker>en0_checker] - en0_checker[en1_checker>en0_checker]) / 0.5)
        boltz_rand = np.random.uniform(0.5, 0.5, size=np.shape(boltz))
        accept += np.sum(boltz >= boltz_rand)

        #creates the index for values that need to be changed back
        index = np.where(checkerboard.reshape(nmax*nmax) == 0)[0][np.where(en1_checker>en0_checker)[0][np.where(boltz<boltz_rand)[0]]]
        print(index)
    
        #changes values back, and changes all values that arent part of the half checkerboard
        arr.reshape(nmax**2)[index] -= aran.reshape(nmax**2)[index]
        arr[checkerboard == (i+1)%2] = arr_copy[checkerboard == (i+1)%2]

    print(arr)
    return accept/(nmax*nmax)





In [571]:
grid = np.mgrid[0:nmax,0:nmax].reshape(2,-1).T
aran = np.random.normal(scale=0.6, size=(nmax,nmax))

In [576]:
arr = np.arange(nmax*nmax, dtype = np.float64).reshape(nmax, nmax)
MC_step(arr, nmax)

[[ 0.          0.37757307  2.07639341  2.91397991]
 [ 3.80301033  5.50512098  6.14992847  6.53662925]
 [ 8.52331322  8.88431054  9.85207715 10.76698409]
 [12.10499102 13.19820401 13.57759592 14.63306389]]


0.9375

In [577]:
arr = np.arange(nmax*nmax, dtype = np.float64).reshape(nmax, nmax)
MC_step_fast(arr,nmax)

[0]
[]
[[ 0.          0.37757307  2.07639341  2.91397991]
 [ 3.80301033  5.50512098  6.14992847  6.53662925]
 [ 8.52331322  8.88431054  9.85207715 10.76698409]
 [12.10499102 13.19820401 13.57759592 14.63306389]]


np.float64(0.9375)

In [469]:
timeit(MC_step_fast(arr, nmax))

241 μs ± 1.2 μs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [ ]:


#My workings to get to my solution

arr = np.arange(nmax*nmax, dtype = np.float64).reshape(nmax, nmax)
arr_orig = arr.copy()
print(arr)
accept = 0
en0 = one_energy(arr, grid[:,0], grid[:,1], nmax)
arr[checkerboard == 0] += aran[checkerboard == 0]
print(arr)
en1 = one_energy(arr, grid[:,0], grid[:,1], nmax)

en0_checker = en0.reshape(nmax, nmax)[checkerboard == 0]
en1_checker = en1.reshape(nmax, nmax)[checkerboard == 0]
#print(en0, en0_checker)
accept += np.sum(en1_checker <= en0_checker)

#adds except for every value of en1 that is smaller than en0
accept += np.sum(en1 <= en0)


boltz_indices = np.where(en1>en0)
boltz = np.exp(-(en1[en1>en0] - en0[en1>en0]) / 0.5)
boltz_rand = np.random.uniform(0.0, 1.0, size=np.shape(boltz))
accept += np.sum(boltz >= boltz_rand)
bolts_indices2 = np.where(boltz<boltz_rand)
print(boltz_indices[0][bolts_indices2[0]])
tot_indices = boltz_indices[0][bolts_indices2[0]]


#arr_flat = arr.reshape(nmax**2)
#aran_flat = aran.reshape(nmax**2)
#arr_flat[tot_indices] -= aran_flat[tot_indices]
#print(arr_flat.reshape(nmax,nmax))
#arr = arr_flat.reshape(nmax, nmax)
print(arr)
arr[checkerboard == 0][tot_indices] -= aran[checkerboard == 0][tot_indices]
#arr[checkerboard == 1] = arr_orig[checkerboard == 1]
#arr[boltz_indices][bolts_indices2] - aran[boltz_indices][bolts_indices2]
print(arr)
print(accept)

[[ 0.  1.  2.  3.]
 [ 4.  5.  6.  7.]
 [ 8.  9. 10. 11.]
 [12. 13. 14. 15.]]
[[ 0.15557532  1.          2.18130094  3.        ]
 [ 4.          4.18138414  6.          6.77914981]
 [ 8.13533018  9.         11.35844165 11.        ]
 [12.         12.60666133 14.         15.03950652]]
[]
[[ 0.15557532  1.          2.18130094  3.        ]
 [ 4.          4.18138414  6.          6.77914981]
 [ 8.13533018  9.         11.35844165 11.        ]
 [12.         12.60666133 14.         15.03950652]]
[[ 0.15557532  1.          2.18130094  3.        ]
 [ 4.          4.18138414  6.          6.77914981]
 [ 8.13533018  9.         11.35844165 11.        ]
 [12.         12.60666133 14.         15.03950652]]
23
